In [39]:
import pandas as pd

df = pd.read_csv("C:\\Users\\HP\\OneDrive\\Desktop\\House_price_prediction\\dataset\\Bengaluru_House_Data.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [40]:
df['bhk'] = df['size'].str.extract('(\d+)').astype(float) # string to numerical

def convert_sqft(x):
    try:
        x = str(x).strip()

        # For intervals
        if '-' in x:
            parts = x.split('-')
            if len(parts) == 2:
                a, b = float(parts[0]), float(parts[1])
                return (a + b) / 2

        # numerical values
        return float(x)

    except:
        # Non-numeric values
        return None
    
df['total_sqft'] = df['total_sqft'].apply(convert_sqft)
df.drop(['area_type', 'society', 'availability', 'balcony'], axis=1, inplace=True)
df.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2.0
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4.0
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3.0
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3.0
4,Kothanur,2 BHK,1200.0,2.0,51.00,2.0


In [43]:
df['price_per_sqft'] = (df['price'] * 1e5) / df['total_sqft']

df = df[df['location'].notnull()]  # Remove rows with null location
df['location'] = df['location'].apply(lambda x: x.strip())
loc_counts = df['location'].value_counts()
rare_locs = loc_counts[loc_counts <= 10].index
df['location'] = df['location'].apply(lambda x: 'other' if x in rare_locs else x)

df.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2.0,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4.0,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3.0,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3.0,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2.0,4250.000000
